In [ ]:
import numpy as np
import os

import pandas as pd 

from pymks.bases import gsh 
from pymks.stats import * 
from pymks import PrimitiveBasis

In [ ]:
def microstructure(filepath, file, x_size, y_size, output=0):
    print(str(file))

    file_with_path = ''.join([str(filepath) + '/' + str(file)])
    ctf_data = pd.read_csv(file_with_path, header = None, skiprows = 1, nrows = 10, sep='\t')

    width = int(ctf_data[1][3])
    height = int(ctf_data[1][4])

    x_space = float(ctf_data[1][5])
    y_space = float(ctf_data[1][6])

    texture = pd.read_csv(file_with_path, header = None, skiprows = 18, sep = '\t', index_col = None, usecols = [0, 5, 6, 7])

    texture = texture.to_numpy()

    def pixel_index(coord):
        return coord[0] + coord[1] * width + 1

    x_min = 0               # x_size = 329
    x_max = x_min + int(x_size/(1 if (x_space > 0.75) else 0.5))
    y_min = 0               # y_size = 370
    y_max = y_min + int(y_size/(1 if (y_space > 0.75) else 0.5))

    texture_rad = np.zeros((x_size*y_size,3))
    phase = np.zeros((x_size*y_size))

    k = 0
    for j in range(y_min,y_max, round(1/y_space)):
        for i in range(x_min,x_max, round(1/x_space)):
            index = pixel_index((i,j))
            texture_rad[k,:] = np.multiply(texture[index-1,1:],np.pi/180)
            phase[k] = texture[index-1,0]
            k += 1 
    
    if (output == 0):
        return texture_rad
    elif (output == 1):
        return phase
    else:
        return None

In [ ]:
filepath = "little_houses_teeth"
files = sorted(os.listdir(filepath))
num_files = np.size(files)

x_size = 950
y_size = 700

microstructure_data = np.concatenate([microstructure(filepath, file, x_size, y_size, 0) for file in files])
phase_data = np.concatenate([microstructure(filepath, file, x_size, y_size, 1) for file in files])

print(np.shape(microstructure_data))
print(np.shape(phase_data))

In [ ]:
GSH_basis = gsh.GSHBasis(n_states=10, domain='cubic')

prim_basis = PrimitiveBasis(n_states=2, domain=[0,4])

microstructure_data = microstructure_data.reshape(num_files,y_size,x_size,1,3)
phase_data = phase_data.reshape(num_files,y_size,x_size,1)


print(np.shape(microstructure_data.reshape(num_files,x_size,y_size,1,3)))
print(np.shape(phase_data.reshape(num_files,x_size,y_size,1,1)))

In [ ]:
stats_texture = correlate(microstructure_data, GSH_basis, periodic_axes=(0,1,2), correlations=[(1,1), (2,2), (3,3), (4,4), (5,5), (6,6), (7,7), (8,8), (9,9), (1,2), (1,3), (1,4), (1,5), (1,6), (1,7), (1,8), (1,9)], gsh_basis=1, gsh_value=0)
print(stats_texture.shape)

In [ ]:
# stats_texture = np.concatenate((np.abs(stats_texture), np.angle(stats_texture)), axis=4)

stats_texture = np.abs(stats_texture)

stats_texture = np.divide((stats_texture - np.mean(stats_texture)), np.std(stats_texture))

print(stats_texture.shape)

In [ ]:
stats_phase = correlate(phase_data, prim_basis, periodic_axes=(0,1,2), correlations=[(0,1),(1,1)], gsh_basis=0, gsh_value=0)

stats_phase = np.divide((stats_phase - np.mean(stats_phase)), np.std(stats_phase))

print(stats_phase.shape)

In [ ]:
stats = np.concatenate((stats_texture, stats_phase), axis=4)

np.shape(stats)
np.save('gsh_statistics.npy', stats)